In [ ]:
import pynq
from pynq import Overlay
import asnycio
from pynq import Interrupt
from pynq import MMIO
import time
import matplotlib.pyplot as plt
import numpy as np 
import copy
import math
from numpy.fft import fft, ifft
import random 
import codecs, json

ol = Overlay("./Overlays/overlay1.bit")

# BRAM
bram_in_L = MMIO(0x40000000, 0x4000 )
bram_in_R = MMIO(0x42000000, 0x4000 )
bram_out_L = MMIO(0x44000000, 0x4000 )
bram_out_R = MMIO(0x46000000, 0x4000 )
bram_exc = MMIO(0x48000000, 0x2000 )
bram_gradient = MMIO(0x4A000000, 0x4000 )
bram_exc_current = MMIO(0x4C000000, 0x4000 )

bram_in_reg_L_0 = bram_in_L.array[0:2048]
bram_in_reg_L_1 = bram_in_L.array[2048:4096]
bram_in_reg_L_0.dtype = np.int32
bram_in_reg_L_1.dtype = np.int32

bram_in_reg_R_0 = bram_in_R.array[0:2048]
bram_in_reg_R_1 = bram_in_R.array[2048:4096]
bram_in_reg_R_0.dtype = np.int32
bram_in_reg_R_1.dtype = np.int32

bram_out_reg_L_0 = bram_out_L.array[0:2048]
bram_out_reg_L_1 = bram_out_L.array[2048:4096]
bram_out_reg_L_0.dtype = np.int32
bram_out_reg_L_1.dtype = np.int32

bram_out_reg_R_0 = bram_out_R.array[0:2048]
bram_out_reg_R_1 = bram_out_R.array[2048:4096]
bram_out_reg_R_0.dtype = np.int32
bram_out_reg_R_1.dtype = np.int32
bram_exc_reg = bram_exc.array
bram_exc_reg.dtype = np.int32
bram_gradient_reg_0 = bram_gradient.array[0:2048]
bram_gradient_reg_1 = bram_gradient.array[2048:4096]
bram_gradient_reg_0.dtype = np.int32
bram_gradient_reg_1.dtype = np.int32
bram_exc_current_reg_0 = bram_exc_current.array[0:2048]
bram_exc_current_reg_1 = bram_exc_current.array[2048:4096]
bram_exc_current_reg_0.dtype = np.int32
bram_exc_current_reg_1.dtype = np.int32

# Interrupts
ol.interrupt_pins
interrupt_0 = Interrupt ('Memory_controller/interrupt_0')
interrupt_1 = Interrupt ('Memory_controller/interrupt_1')

# Define ISR
gradient_data_in = np.full(2048, 0)
exc_current_data_in = np.full(2048, 0)
sensor_data_in_1 = np.full(2048, 0)
sensor_data_in_2 = np.full(2048, 0)
async def handle_readData():
    await interrupt_0.wait()
    np.copyto(gradient_data_in, bram_gradient_reg_0)
    np.copyto(exc_current_data_in, bram_exc_current_reg_0)
    np.copyto(sensor_data_in_1, bram_in_reg_L_0)
    np.copyto(sensor_data_in_2, bram_in_reg_R_0)


asyncio.ensure_future(handle_readData())

# Create Fourier Components
def fullSpectrum(freq_spectrum):
    spectrum_mirror = np.full(1,0,dtype = complex)
    spectrum_mirror = np.append(spectrum_mirror, np.flip(np.conj(freq_spectrum [1:])))

    return np.concatenate((freq_spectrum, spectrum_mirror))

def generateSignal(frequency_spectrum_exc):
    return ifft(fullSpectrum(frequency_spectrum_exc)).real


#Generate signal with mixed frequencies
amplitude_spectrum_exc = np.full(1024,0)

for i in np . arange (1 ,60) :
    amplitude_spectrum_exc[i] = 2048

phase_spectrum_exc = np.full(1024 ,0)
for i in np.arange(1,60):
    phase_spectrum_exc[i] = random.uniform(0, 2*math.pi)

frequency_spectrum_exc = amplitude_spectrum_exc*(np.cos (phase_spectrum_exc) + np.sin(phase_spectrum_exc) * 1j)

exc_signal = generateSignal(frequency_spectrum_exc)

# Generate Signal with correct Amplitude
amplitudeFactor = 0x27FF / np.amax(exc_signal) # Max Amp . 0x27FF
exc_signal = np.int32(np.multiply(amplitudeFactor, exc_signal))
print(exc_signal)

#A function to set the amplitude of the excitation coil
def setAmplitude(factor, exc_signal):
    ##Scaling Amplitude of signal
    exc_signal = np.multiply(factor, exc_signal)
    np.copyto(bram_exc_reg, np.int32(exc_signal))

def readFlux1():
    return sensor_data_in_1

def readFlux2():
    return sensor_data_in_2

def readGradientFlux():
    return gradient_data_in

def readExcCurrent():
    return exc_current_data_in

def measurement(number): 
    flux1 = np.full(2048,0,dtype=np.float32)
    flux2 = np.full(2048,0,dtype=np.float32)
    gradient_flux = np.full(2048,0,dtype=np.float32)
    exc_current = np.full(2048,0,dtype=np.float32)

    for i in range(number): 
        asyncio.get_event_loop().run_until_complete(handle_readData())
        #print("interrupt")
        flux1_t = readFlux1()
        flux2_t = readFlux2()
        gradient_flux_t = readGradientFlux()
        exc_current_t = readExcCurrent()
        flux1 += flux1_t.astype(float)
        flux2 += flux2_t.astype(float)
        gradient_flux += gradient_flux_t.astype(float)
        exc_current += exc_current_t.astype(float)

    flux1 /= number
    flux2 /= number
    gradient_flux = /= number
    exc_current /= number

    return flux1, flux2, gradient_flux, exc_current

def fourierTransform(flux1, flux2, gradient_flux, exc_current):
    n = np.arange(0,1024)
    frequencies = n*48000/2048
    return frequencies, fft(flux1), fft(flux2), fft(gradient_flux), fft(exc_current)

# A function to calibrate the sensors with the current magnetic field induced by the excitations coil
def setReference(number):
    flux1_ref = np.full(2048, 0, dtype = np.float32)
    flux2_ref = np.full(2048, 0, dtype = np.float32)
    gradient_flux_ref = np.full(2048, 0, dtype = np.float32)
    flux1_ref, flux2_ref, gradient_flux_ref, _ = measurement(number)

    return flux1_ref, flux2_ref, gradient_flux_ref

# Save the data to a JSON file
def complexToDict(z):
    return { "real" : z.real, "imag" : z.imag}
def saveAsJson(complex_array, file_path):
    serializable_array = [[complexToDict (z) for z in row] for row in complex_array]
    json.dump(serializable_array, codecs.open(file_path," w ",encoding = " utf -8 " ) , separators =( " ," , " : " ),sort_keys = True, indent =4)
setAmplitude(4.5, exc_signal)
flux1_ref, flux2_ref, gradient_flux_ref = setReference(300)
print(flux1_ref)
print(flux2_ref)

# define arrays
c_ring_4_1 = np.full((10,2048),0,dtype=complex)
c_ring_6_05 = np.full((10,2048),0,dtype=complex)
c_ring_8_55 = np.full((10,2048),0,dtype=complex)
c_ring_13_75 = np.full((10,2048),0,dtype=complex)
sss = np.full((10,2048),0,dtype=complex)
ssm = np.full((10,2048),0,dtype=complex)
ssb = np.full((10,2048),0,dtype=complex)
distance = ["12.5","17.5","21.5","25.5","29.5","33.5","37.5","41.5","45.5","49.5"]

# Function to do one measurement after calibration
def getSpectrum(number, name, distance):
    lim = 45
    flux1, flux2, gradient_flux, exc_current = measurement(number)
    # flux1 -= flux1_ref
    # flux2 -= flux2_ref
    gradient_flux -= gradient_flux_ref
    frequencies, flux1_f, flux2_f, gradient_flux_f, exc_current_f = fourierTransform(flux1, flux2, gradient_flux, exc_current)
    ratio = flux2_f / flux1_f
    ratio -= fft(flux2_ref) / fft(flux1_ref)
    fig,axs = plt.subplots(2,2,figsize=(15,9))
    fig.suptitle(f"{name} at distance = {distance}cm")
    axs[0,0].plot(frequencies[1:lim],ratio.real[1:lim])
    axs[0,0].set_title("real part")
    axs[0,1].plot(frequencies[1:lim],ratio.imag[1:lim])
    axs[0,1].set_title("imaginary part")
    axs[1,0].plot(frequencies[1:lim],np.abs(ratio)[1:lim])
    axs[1,0].set_title("magnitude")
    axs[1,1].plot(frequencies[1:lim],np.angle(ratio)[1:lim])
    axs[1,1].set_title("phase")
    fig.subplots_adjust(wspace=0.5,hspace=0.5)
    for ax in axs.flat:
        ax.grid(True)
    plt.show()
    return ratio

# Measurement for copper ring with radius 4.1 cm
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    c_ring_4_1[i] = getSpectrum(300, "copper ring 4.1 cm radius", distance[i])
print("Measurement done")

# Measurement for copper ring with radius 6.05 cm
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    c_ring_6_05[i] = getSpectrum(300, "copper ring 6.05 cm radius", distance[i])
print("Measurement done")

# Measurement for copper ring with radius 8.55 cm
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    c_ring_8_55[i] = getSpectrum(300, "copper ring 8.55 cm radius", distance[i])
print("Measurement done")

# Measurement for copper ring with radius 13.75 cm
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    c_ring_13_75[i] = getSpectrum(300, "copper ring 13.75 cm radius", distance[i])
print("Measurement done")

# Measurement for stainless steel small
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    sss[i] = getSpectrum(300, "stainless steel small", distance[i])
print("Measurement done")

# Measurement for stainless steel middle
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    ssm[i] = getSpectrum(300, "stainless steel middle", distance[i])
print("Measurement done")

# Measurement for stainless steel big
for i in range(10):
    for k in range(15):
        print(15 - k)
        time.sleep(1)
    ssb[i] = getSpectrum(300, "stainless steel big", distance[i])
print("Measurement done")

saveAsJson(c_ring_4_1, "./c_ring_4_1.json")
saveAsJson(c_ring_6_05, "./c_ring_6_05.json")
saveAsJson(c_ring_8_55, "./c_ring_8_55.json")
saveAsJson(c_ring_13_75, "./c_ring_13_75.json")
saveAsJson(sss, "./sss.json")
saveAsJson(ssm, "./ssm.json")
saveAsJson(ssb, "./ssb.json")